In [ ]:
import arcpy
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

## Inputs

In [ ]:
taz_shp = '.\\Inputs\\TAZ.shp'
parcels = '.\\Inputs\\Utah_Weber_County_Parcels_LIR.gdb\\Parcels_Weber_LIR_UTM12'
address_pts = '.\\Inputs\\AddressPoints.gdb\\AddressPoints_Weber'
common_areas = '.\\Inputs\\Weber_Owned_Unit_Groupings.gdb\\Weber_Owned_Unit_Groupings'

## Create geodatabases for output files

In [ ]:
# create output gdb
outputs = '.\\Outputs'
gdb = os.path.join(outputs, "classes.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "classes.gdb")
    
scratch = os.path.join(outputs, "scratch.gdb")
if not arcpy.Exists(scratch):
    arcpy.CreateFileGDB_management(outputs, "scratch.gdb")
    
final = os.path.join(outputs, "weber.gdb")
if not arcpy.Exists(final):
    arcpy.CreateFileGDB_management(outputs, "weber.gdb")

## Filter Address points (used later)

In [ ]:
# get address points without base or commercial points
address_pts_lyr = arcpy.MakeFeatureLayer_management(address_pts,'address_pts_lyr')
query = (''' PtType = 'Residential' ''')
arcpy.SelectLayerByAttribute_management(address_pts_lyr, 'NEW_SELECTION', query)
address_pts_no_base = arcpy.FeatureClassToFeatureClass_conversion(address_pts_lyr, scratch, '_00_address_pts_no_base')

## Prep main parcel layer

In [ ]:
# select parcels within modeling area
parcels_layer = arcpy.MakeFeatureLayer_management(parcels, 'parcels') 
arcpy.SelectLayerByLocation_management(parcels_layer, 'HAVE_THEIR_CENTER_IN', taz_shp)
parcels_for_modeling = arcpy.FeatureClassToFeatureClass_conversion(parcels_layer, scratch, '_01_parcels_for_modeling')

# recalc acreage 
arcpy.CalculateField_management(parcels_for_modeling, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

# create the main layer
parcels_for_modeling_layer = arcpy.MakeFeatureLayer_management(parcels_for_modeling, 'parcels_for_modeling_lyr') 

# house count field is a string for some reason, so this is the fix
query = """ (HOUSE_CNT IS NULL) OR (HOUSE_CNT = ' ') """
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
arcpy.CalculateField_management(parcels_for_modeling_layer, "HOUSE_CNT", """0""", "PYTHON3")
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
arcpy.AddField_management(parcels_for_modeling_layer, 'HOUSE_CNT2', 'LONG')
arcpy.CalculateField_management(parcels_for_modeling_layer, "HOUSE_CNT2", """int(!HOUSE_CNT!)""", "PYTHON3")
arcpy.DeleteField_management(parcels_for_modeling_layer,"HOUSE_CNT")

# add second built year field
arcpy.AddField_management(parcels_for_modeling_layer, 'BUILT_YR2', 'SHORT')
arcpy.CalculateField_management(parcels_for_modeling_layer, "BUILT_YR2", """!BUILT_YR!""", "PYTHON3")

In [ ]:
################################
# Dissolve duplicate parcel ids
################################

# dissolve on parcel id,summarizing attributes in various ways
parcels_dissolved = arcpy.management.Dissolve(parcels_for_modeling_layer, os.path.join(scratch, '_00_parcels_dissolved'), 
                                              "PARCEL_ID", [['PARCEL_ID','COUNT'],['COUNTY_NAME','FIRST'],
                                                            ['COUNTY_ID','FIRST'],['TAXEXEMPT_TYPE','FIRST'],
                                                            ['SUBDIV_NAME','FIRST'],
                                                            ['TOTAL_MKT_VALUE','MAX'],['LAND_MKT_VALUE','MAX'],
                                                            ['PARCEL_ACRES','SUM'],['PROP_CLASS', 'FIRST'],
                                                            ['PRIMARY_RES','FIRST'],['HOUSE_CNT2','MAX'],
                                                            ['BLDG_SQFT','MAX'],['FLOORS_CNT','MAX'], ['BUILT_YR','MAX'],
                                                            ['BUILT_YR2','MAX'],
                                                            ['EFFBUILT_YR','FIRST']], 
                                              "MULTI_PART", "DISSOLVE_LINES")

# rename columns
parcels_dissolved_sdf = pd.DataFrame.spatial.from_featureclass(parcels_dissolved[0])
parcels_dissolved_sdf.columns = ['OBJECTID', 'PARCEL_ID', 'COUNT_PARCEL_ID','COUNTY_NAME','COUNTY_ID','TAXEXEMPT_TYPE',
                                 'SUBDIV_NAME','TOTAL_MKT_VALUE','LAND_MKT_VALUE', 'PARCEL_ACRES','PROP_CLASS',
                                 'PRIMARY_RES','HOUSE_CNT','BLDG_SQFT', 'FLOORS_CNT', 'BUILT_YR','BUILT_YR2','EFFBUILT_YR', 
                                 'SHAPE']

# remove parcels without parcel ids
parcels_dissolved_sdf = parcels_dissolved_sdf[parcels_dissolved_sdf['PARCEL_ID'].isnull() == False].copy()

# export and recreate layer
parcels_dissolved_sdf.spatial.to_featureclass(location=os.path.join(scratch, '_01_parcels_dissolved'),
                                              sanitize_columns=False)
updated_parcels = os.path.join(scratch, '_01_parcels_dissolved')
parcels_for_modeling_layer = arcpy.MakeFeatureLayer_management(updated_parcels, 'parcels_for_modeling_lyr')

## More parcel prep

In [ ]:
# add a tag field for parcel type
arcpy.AddField_management(parcels_for_modeling_layer, 'TYPE_WFRC', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'SUBTYPE_WFRC', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'NOTE', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'IS_OUG', 'SHORT')

# add REMM fields
arcpy.AddField_management(parcels_for_modeling_layer, 'zone_id', 'LONG')
arcpy.AddField_management(parcels_for_modeling_layer, 'elevation', 'FLOAT')
arcpy.AddField_management(parcels_for_modeling_layer, 'max_dua', 'FLOAT')
arcpy.AddField_management(parcels_for_modeling_layer, 'max_far', 'FLOAT')
arcpy.AddField_management(parcels_for_modeling_layer, 'max_height', 'FLOAT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type1', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type2', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type3', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type4', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type5', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type6', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type7', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type8', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'agriculture', 'SHORT')
arcpy.AddField_management(parcels_for_modeling_layer, 'basebldg', 'SHORT')
arcpy.AddField_management(parcels_for_modeling_layer, 'NoBuild', 'SHORT') # REMM doesn't use this
arcpy.AddField_management(parcels_for_modeling_layer, 'redev_friction', 'FLOAT')
arcpy.AddField_management(parcels_for_modeling_layer, 'building_type_id', 'SHORT')

fields=['basebldg','type1','type2','type3','type4','type5','type6','type7','type8','agriculture','NoBuild']
with arcpy.da.UpdateCursor(parcels_for_modeling_layer, fields) as cursor:
    for row in cursor:
        row[0] = 0
        row[1] = 'f'
        row[2] = 'f'
        row[3] = 'f'
        row[4] = 'f'
        row[5] = 'f'
        row[6] = 'f'
        row[7] = 'f'
        row[8] = 'f'
        row[9] = 0
        row[10] = 0
        cursor.updateRow(row)

# count parts and rings
arcpy.AddField_management(parcels_for_modeling_layer, 'PARTS', 'LONG')
arcpy.AddField_management(parcels_for_modeling_layer, 'RINGS', 'LONG')

fields = ["shape@", 'PARTS', 'RINGS']
with arcpy.da.UpdateCursor(parcels_for_modeling_layer, fields) as cursor:
    for row in cursor:
        shape = row[0]
        parts = shape.partCount
        rings = shape.boundary().partCount   
        row[1] = parts
        row[2] = rings
        cursor.updateRow(row)      


# delete parcels with empty geometry
query = (""" PARTS =  0""")
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# get a count of all parcels
count_all = arcpy.GetCount_management(parcels_for_modeling_layer)
print("# initial parcels in modeling area:\n {}".format(count_all))

## Load Owned Unit Groupings boundaries (common areas) and summarize parcels within

In [ ]:
###############
# Common Areas
###############

common_areas_lyr = arcpy.MakeFeatureLayer_management(common_areas, 'common_areas_lyr') 

# open_space
query = """ TYPE_WFRC IN ('open_space') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_open_space = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02a_ca_open_space')

# vacant
query = """ TYPE_WFRC IN ('vacant') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_vacant = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02b_ca_vacant')


# commercial
query = """ TYPE_WFRC IN ('commercial') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_commercial = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02d_ca_commercial')
arcpy.CalculateField_management(ca_commercial, field='IS_OUG', expression="1",expression_type="PYTHON3")

# pud
query = """ SUBTYPE_WFRC IN ('pud') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_pud = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02h_ca_pud')
arcpy.CalculateField_management(ca_pud, field='IS_OUG', expression="1",expression_type="PYTHON3")

# multi_family
query = """ TYPE_WFRC IN ('multi_family') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_multi_family = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02i_ca_multi_family')
arcpy.CalculateField_management(ca_multi_family, field='IS_OUG', expression="1",expression_type="PYTHON3")

In [ ]:
###############
# PUD
###############

tag = "single_family"
tag2 = "pud"

# parcels that are contained by condo common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=ca_pud, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
pud_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03a_pud_centroids'), "INSIDE")

# recalc acreage
arcpy.CalculateField_management(ca_pud, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within pud areas
#==================================================

# use spatial join to summarize market value & acreage
target_features = ca_pud
join_features = pud_centroids
output_features = os.path.join(scratch, "_03a_pud_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# total land value
fieldindex = fieldmappings.findFieldMapIndex('LAND_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# house count
fieldindex = fieldmappings.findFieldMapIndex('HOUSE_CNT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'SUM'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# building square feet
fieldindex = fieldmappings.findFieldMapIndex('BLDG_SQFT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# floor count
fieldindex = fieldmappings.findFieldMapIndex('FLOORS_CNT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mean'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# year
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# built year max
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR2')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

arcpy.CalculateField_management(oug_sj, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!",
                                expression_type="PYTHON3")

arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ADDR_CNT"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_pud")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ADDR_CNT', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# update year built with max if mode is 0
with arcpy.da.UpdateCursor(oug_sj2, ['BUILT_YR', 'BUILT_YR2']) as cursor:
    for row in cursor:
        if row[0]  is None or row[0] < 1 or row[0] == "":
            row[0] = row[1]
        
        cursor.updateRow(row)
        
# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="1",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
###############
# multi family
###############

tag = "multi_family"

# parcels that are contained by condo common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=ca_multi_family, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
mf_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03a_mf_centroids'), "INSIDE")

# recalc acreage
arcpy.CalculateField_management(ca_multi_family, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within pud areas
#==================================================

# use spatial join to summarize market value & acreage
target_features = ca_multi_family
join_features = mf_centroids
output_features = os.path.join(scratch, "_03b_mf_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# total land value
fieldindex = fieldmappings.findFieldMapIndex('LAND_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# building square feet
fieldindex = fieldmappings.findFieldMapIndex('BLDG_SQFT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# house count
fieldindex = fieldmappings.findFieldMapIndex('HOUSE_CNT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'SUM'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# floor count
fieldindex = fieldmappings.findFieldMapIndex('FLOORS_CNT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mean'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# year
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# built year max
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR2')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!",
                                expression_type="PYTHON3")

arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ADDR_CNT"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_multi_family")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ADDR_CNT', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# update year built with max if mode is 0
with arcpy.da.UpdateCursor(oug_sj2, ['BUILT_YR', 'BUILT_YR2']) as cursor:
    for row in cursor:
        if row[0]  is None or row[0] < 1 or row[0] == "":
            row[0] = row[1]
        
        cursor.updateRow(row)

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="2",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
#####################
# commercial commons
#####################

tag = "commercial"

# parcels that are contained by industrial common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=ca_commercial, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
commercial_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03a_commercial_centroids'), "INSIDE")

# recalc acreage
arcpy.CalculateField_management(ca_commercial, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within pud areas
#==================================================

# use spatial join to summarize market value & acreage
target_features = ca_commercial
join_features = commercial_centroids
output_features = os.path.join(scratch, "_03e_commercial_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# total land value
fieldindex = fieldmappings.findFieldMapIndex('LAND_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# building square feet
fieldindex = fieldmappings.findFieldMapIndex('BLDG_SQFT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# house count
fieldindex = fieldmappings.findFieldMapIndex('HOUSE_CNT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'SUM'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# floor count
fieldindex = fieldmappings.findFieldMapIndex('FLOORS_CNT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mean'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# year
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# built year max
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR2')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!",
                                expression_type="PYTHON3")

arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ADDR_CNT"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_commercial_commons")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ADDR_CNT', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# update year built with max if mode is 0
with arcpy.da.UpdateCursor(oug_sj2, ['BUILT_YR', 'BUILT_YR2']) as cursor:
    for row in cursor:
        if row[0]  is None or row[0] < 1 or row[0] == "":
            row[0] = row[1]
        
        cursor.updateRow(row)

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="5",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

## Categorize parcels

In [ ]:
###########
# churches
###########

query = (""" TAXEXEMPT_TYPE IN ('801-LDS Church','803-Baptist Church', '804-Buddhist Church', '805-Catholic Church', 
            '806-Other Denominations') OR PARCEL_ID IN ('240290002','240290003') """) 
tag="churches"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="10",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
###############
# agriculture
###############

query = """ PROP_CLASS IN ('811- FARM ANIMALS','812- PROD FARM') """
tag="agriculture"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="15",
                                expression_type="PYTHON3")

# calculate agriculture field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='agriculture', expression="1",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
###############
# education
###############

query = (""" TAXEXEMPT_TYPE IN ('310-Ogden City Schools','320-Weber County Schools','340-WSU') OR 
             PARCEL_ID IN ('130260001','130050091') """)
tag="education"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="9",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)


# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
################
# single_family
################

query = (''' PROP_CLASS IN ('108- SFR on COMM Zone', '111- SNGL FAM RES', '502-COMM LAND with SFR', 
            '508- Both RES & COMM IMPRVD') OR PARCEL_ID IN ('161300020', '010600021','130340037') ''')
tag="single_family"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# calulate building field to "yes"

# # these parcels are probably sf residential, but with no building
# query = (''' PROP_CLASS IN ('121-OUT BLDGS', '957-RELATED PARCEL') OR PARCEL_ID IN ('130170033') ''')
# arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
single_family = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="1",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
################
# Duplex
################

tag="multi_family"
tag2="duplex"

query = (''' PROP_CLASS IN ('112- DUPLEX') ''')

arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)


# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag2))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="2",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
################
# Apartment
################

query = (''' PROP_CLASS IN ('113- 3-4 UNITS','114- 5-9 UNITS', '115- 10+ UNITS') OR 
            PARCEL_ID IN ('010600021', '122210002','120840030', '120310022')''')
tag="multi_family"
tag2="apartment"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
mf2_commons = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_02_mf2_commons')

#################################
# get count from address points
#################################

# summarize address points address_point_count "ADDR_CNT"
target_features = mf2_commons
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_apartment")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ADDR_CNT', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="2",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
#################
# group quarters
#################

query = (''' PROP_CLASS IN ('300- MULTIPLE HOUSING') ''')
tag = "group_quarters"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="11",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
#################
# mobile home parks
#################

tag = "multi_family"
tag2 = "mobile_home_park"

# use overlay to select mobile home parks parcels
mobile_home_parks = ".\\Inputs\\Mobile_Home_Parks.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=mobile_home_parks,
                                       selection_type='NEW_SELECTION')

query= (""" PROP_CLASS IN ('118- MOBILE HOME') """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
mhp = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_07a_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# recalc acreage 
arcpy.CalculateField_management(mhp, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ADDR_CNT"
target_features = mhp
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_mobile_home_park")

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ADDR_CNT', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")
# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="2",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
###############
# utilities
###############

query= (""" TAXEXEMPT_TYPE IN ('601-Mountain Fuel (Questar)','602-Utah Power and Light','603-US West Telephone',
            '604-All Other Utilities', '605-Weber Basin Water','607-Water Tank Parcel','609-Water & Sewer Districts', 
            '995-Taxable Water Companies') """)

tag="undevelopable"
tag2="utilities"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
features= arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate building_type_id field
arcpy.CalculateField_management(features, field='building_type_id', expression="16",
                                expression_type="PYTHON3")

# calculate no_build field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='NoBuild', expression="1",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
###############
# health care
###############

query = (""" (SUBDIV_NAME LIKE '%HOSP%' OR SUBDIV_NAME LIKE '%CLINIC%' OR 
              SUBDIV_NAME LIKE '%MEDICAL%') OR (PARCEL_ID IN ('076590005','076590006','076590007','070530072',
                '070530069','070530036','070530054')) """)
tag = 'healthcare'

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# add parcels that overlap with healthcare points - currently excluding nursing homes
healthcare_facilties_lyr = arcpy.MakeFeatureLayer_management('.\\Inputs\\HealthCareFacilities.shp','healthcare_facilties_lyr') 
query = """ TYPE NOT IN ('NURSING HOME', 'ASSISTED LIVING FACILITY') """
arcpy.SelectLayerByAttribute_management(healthcare_facilties_lyr, 'NEW_SELECTION', query)
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",
                                       select_features=healthcare_facilties_lyr,
                                       selection_type='ADD_TO_SELECTION')


# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="13",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
###############
# government
###############

query = (""" TAXEXEMPT_TYPE IN ('100-Federal Government','110-USA','200-State of Utah','400-Weber County', 
            '450-Weber County Library','501-Ogden City','502-South Ogden City','503-Ogden City Redevelopment','505-Roy City','506-Marriott-Slaterville',
            '507-North Ogden City','508-Farr West City','509-Pleasant View City','510-Plain City','511-Harrisville City',
            '512-Riverdale City','513-Uintah (City of)','514-West Haven City','515-Wash Terrace City','516-Hooper City',
            '517-South Weber (City of)','800-Charitable Organization','010210067') """)
tag="government"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)


# get parcels on hill afb
hill_afb = ".\\Inputs\\hill_afb.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",
                                       select_features=hill_afb,
                                       selection_type='ADD_TO_SELECTION')

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="6",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
###############
# industrial
###############

query = (""" PROP_CLASS IN ('200-INDUSTRIAL') or PARCEL_ID IN ('070820020','070820021','070820028','070820049',
                                                               '070820025','070820019','051260032','051250010',
                                                               '051250009','051250068','051250067','040490027',
                                                               '040470006','040480026','040470005','040430023',
                                                               '040480028','040480024','051250073','040430007',
                                                               '040430008','040490032','040470012','040430022',
                                                               '051250053','051250046','040430012','040460047',
                                                               '040490007','040430051','040470007','040430014',
                                                               '051250047','051250049','040480023','040430011',
                                                               '040460031','040430021','040430029','040430050',
                                                               '070520061','070520062','070520059','070520058',
                                                               '070520055','030080018','030360007','121270002',
                                                               '121270003','121240027','121240022','121250050',
                                                               '114340001','170660038','170660013','190410029',
                                                               '190410028','190410026','190640006','190640004',
                                                               '190640003','110200072','110200077','110200076',
                                                               '114090009','114090008','110200040','114090007',
                                                               '121250050')""")
tag="industrial"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="3",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
###############
# commercial/retail
###############

query = (""" PROP_CLASS IN ('500- COMMERCIAL') or PROP_CLASS IN ('952- EXEMPT - COM') OR PARCEL_ID IN ('051410006',
             '051410001','051410019','051380025','051380024','051380023','051380026','051380027','051380030',
             '051380029','051380028','051400008','051400020','051400004','051400010','051400014','051400018',
             '051390077','051250022','051210030','051250001','051240007','121080015','121080019','030450002',
             '051390078','051390090','051390086','051390081','051390064','051960003','060300045','070520059',
             '040260013','040260015','040260041','040290030','040290029','040410063','040410003','113520001') """)

tag="commercial_retail"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
features = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(features, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(features, field='building_type_id', expression="4",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
#################
# roads/corridors
#################

query=""" (TAXEXEMPT_TYPE IN ('210-State Road Comm','402-Weber Pathways','611-Private Road','900-Railroad Properties')) OR 
          (PARCEL_ID IN ('170650033')) """
tag="roads"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
features = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))


# calculate building_type_id field
arcpy.CalculateField_management(features, field='building_type_id', expression="99",
                                expression_type="PYTHON3")

# calculate no_build field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='NoBuild', expression="1",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
###############
# open space
###############

# get parcels on parks
parks = ".\\Inputs\\ParksLocal.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=parks,
                                       selection_type='NEW_SELECTION')

# get parcels on public lands that are not military
land_ownership_lyr = arcpy.MakeFeatureLayer_management('./Inputs/UT_SITLA_Ownership_LandOwnership_WM.shp','land_ownership_lyr') 

query= (""" OWNER <> 'Private' And DESIG <> 'Military' """)
arcpy.SelectLayerByAttribute_management(land_ownership_lyr, 'NEW_SELECTION', query)

arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=land_ownership_lyr,
                                       selection_type='ADD_TO_SELECTION')

# get cemetery parcels using overlay
cemeteries_lyr = arcpy.MakeFeatureLayer_management('.\\Inputs\\Cemeteries.shp','cemeteries_lyr') 
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",
                                       select_features=cemeteries_lyr,
                                       selection_type='ADD_TO_SELECTION')

# get parcels that are golf courses using overlay
golf_courses = ".\\Inputs\\GolfCourses.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=golf_courses,
                                       selection_type='ADD_TO_SELECTION')

#=====================================================
# get parcels that overlap with reviewed common spaces
#======================================================

# open
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=ca_open_space,
                                       selection_type='ADD_TO_SELECTION')

arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="WITHIN",
                                       select_features=ca_open_space,
                                       selection_type='ADD_TO_SELECTION')

#======================================================
# query parcel attributes
#======================================================

query= (""" TAXEXEMPT_TYPE IN ('961-Cemetery') OR PARCEL_ID IN ('010580051','052090002','052090004','050270021',
                                                                '060600001','061680004','060820001','060820003',
                                                                '132030007','131130027','131140017','130870003',
                                                                '131140018','131130015','132030014','131130013',
                                                                '131130021','131130001','130800003','130870003',
                                                                '132030013','131130017','130730014','130720039',
                                                                '130720032','130720033','130720019','130390037',
                                                                '130390019','130720068','130390036','130720068') """)
tag = "open_space"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
features = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate building_type_id field
arcpy.CalculateField_management(features, field='building_type_id', expression="14",
                                expression_type="PYTHON3")

# calculate no_build field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='NoBuild', expression="1",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
#################
# Undevelopable
#################

tag = "undevelopable"

query= (""" PARCEL_ID IN ('010520014', '010520007', '010520004', '010520008', '010520006','010520005', 
            '171410001','154590007') OR PROP_CLASS IN ('121-OUT BLDGS', '957-RELATED PARCEL') """)

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
features = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate building_type_id field
arcpy.CalculateField_management(features, field='building_type_id', expression="99",
                                expression_type="PYTHON3")

# calculate no_build field
arcpy.CalculateField_management(features, field='NoBuild', expression="1",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
####################
# Drop small parcels
####################

tag = 'deleted_small_parcels'
query = """ Shape_Area < 375 """

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_010_{}'.format(tag))

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

arcpy.DeleteFeatures_management(parcels_for_modeling_layer)
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)
print('{} small parcels were deleted.\n{} parcels remain...'.format(count_type, count_remaining))

In [ ]:
#################
# Vacant land
#################

# query= (""" PROP_CLASS IN ('901- VAC - RES - DEV','902- C/I VACANT','911- RES LOT', '917- REC-VAC-LOT', '922- PUD LOT',
#             '999- UNDEVELOPED VL') """)
tag = "vacant"

# select the features
# arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
#                                        select_features=ca_vacant,
#                                        selection_type='ADD_TO_SELECTION')

# arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="WITHIN",
#                                        select_features=ca_vacant,
#                                        selection_type='ADD_TO_SELECTION')

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(features, field='building_type_id', expression="0",
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)


# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [ ]:
# # export remaining parcels
# arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_09_unclassed_parcels')

In [ ]:
##################################
# Merge the parcels back together
##################################

arcpy.env.workspace = gdb
featureclasses = arcpy.ListFeatureClasses()
merged_parcels = arcpy.Merge_management(featureclasses, os.path.join(scratch, 'merged_parcels'),
                                        add_source='ADD_SOURCE_INFO')
merged_parcels_lyr = arcpy.MakeFeatureLayer_management(merged_parcels, 'merged_parcels_lyr') 

## Use the Generalized Future Land Use layer to set the parcel zoning

In [ ]:
# set land use type on parcels using glfu
gflu = '.\Inputs\FutureLandUse2020.gdb\FutureLandUse2020'
gflu_lyr = arcpy.MakeFeatureLayer_management(gflu, 'gflu_lyr') 


##################
# Type 1 (SF)
##################

# query for land use type
query = """ GenLUType IN ('Any Development','Any Residential', 'Mixed Use SF', 'Residential SF', 'Residential SF/Retail') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type1', expression="'t'".format(tag),
                                expression_type="PYTHON3")


##################
# Type 2 (MF)
##################

# query for land use type
query = """ GenLUType IN ('Any Residential', 'Any Development', 'Industrial/Mixed Use MF', 'Mixed Use', 
                          'Residential MF/Office', 'Residential MF', 'Mixed Use MF', 'Mixed Use SF', 'Residential/Office', 
                          'Residential/Retail') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type2', expression="'t'".format(tag),
                                expression_type="PYTHON3")

#######################
# Type 3 (Industrial)
#######################

# query for land use type
query = """ GenLUType IN ('Industrial', 'Any Development', 'Industrial/Mixed Use MF') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type3', expression="'t'".format(tag),
                                expression_type="PYTHON3")

#######################
# Type 4 (Retail)
#######################

# query for land use type
query = """ GenLUType IN ('Any Development', 'Industrial/Retail', 'Retail', 'Retail/Office', 'Residential/Retail', 
                          'Residential SF/Retail') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type4', expression="'t'".format(tag),
                                expression_type="PYTHON3")


#######################
# Type 5 (Office)
#######################

# query for land use type
query = """ GenLUType IN ('Retail/Office', 'Office', 'Any Commercial', 'Residential/Office', 'Residential MF/Office') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type5', expression="'t'".format(tag),
                                expression_type="PYTHON3")

####################################
# Type 6 (Government and Education)
###################################

# query for land use type
query = """ GenLUType IN ('Any Development', 'Government/Education') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type6', expression="'t'".format(tag),
                                expression_type="PYTHON3")


####################################
# Type 7 (Mixed Use)
###################################

# query for land use type
query = """ GenLUType IN ('Any Development', 'Industrial/Mixed Use MF', 'Mixed Use', 'Mixed Use MF', 'Mixed Use SF', 
                          'Residential MF/Office', 'Residential SF/Retail', 'Residential/Office', 'Residential/Retail', 
                          'Retail/Office') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type7', expression="'t'".format(tag),
                                expression_type="PYTHON3")

####################################
# Type 8 (Other)
###################################

# # query for land use type
# query = """ GenLUType IN ('Any Development', 'Industrial/Retail', 'Retail', 'Retail/Office', 'Residential/Retail', 
#                           'Residential SF/Retail') """
# arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# # select the parcels
# arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
#                                        select_features=gflu_lyr,
#                                        selection_type='NEW_SELECTION')

# arcpy.CalculateField_management(merged_parcels_lyr, field='type8', expression="'t'".format(tag),
#                                 expression_type="PYTHON3")

#################################
# Undevelopable
#################################

# query for land use type
query = """ GenLUType IN ('NoBuild') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr, overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

query = """ NoBuild = 1 """
arcpy.SelectLayerByAttribute_management(merged_parcels_lyr, 'NEW_SELECTION', query)


# set all types to false
arcpy.CalculateField_management(merged_parcels_lyr, field='type1', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type2', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type3', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type4', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type5', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type6', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type7', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type8', expression="'f'".format(tag),
                                expression_type="PYTHON3")

# clear selected parcels
arcpy.SelectLayerByAttribute_management(merged_parcels_lyr, 'CLEAR_SELECTION', query)
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'CLEAR_SELECTION', query)

In [ ]:
#######################################################
# run spatial join to get maxdua and maxfar from gflu
#######################################################

target_features = merged_parcels_lyr
join_features = gflu_lyr
output_features = os.path.join(scratch, "_08_parcel_glfu_join")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="HAVE_THEIR_CENTER_IN")


parcel_glfu_join = pd.DataFrame.spatial.from_featureclass(oug_sj2[0])[['OBJECTID', 'MaxDUA']].copy()

## Eject buildings from parcel data

In [ ]:
parcels_buildings_sdf = pd.DataFrame.spatial.from_featureclass(merged_parcels[0])

####################
# drop empty parcels
####################

# parcels_buildings_sdf = parcels_buildings_sdf[parcels_buildings_sdf['PARCEL_ID'].isna() == False]

#######################
# add max dua
#######################

parcels_buildings_sdf = parcels_buildings_sdf.merge(parcel_glfu_join, left_on='OBJECTID', right_on='OBJECTID', how='inner')
parcels_buildings_sdf['max_dua'] = parcels_buildings_sdf['MaxDUA']
del parcels_buildings_sdf['MaxDUA']

#######################
# Fix the unit counts
#######################

# calc note column using property class
parcels_buildings_sdf.loc[(parcels_buildings_sdf['NOTE'].isnull() == True ), 'NOTE'] = parcels_buildings_sdf['PROP_CLASS']

# convert unit count columns to int
parcels_buildings_sdf['HOUSE_CNT'] = 0
parcels_buildings_sdf['HOUSE_CNT'] = parcels_buildings_sdf['HOUSE_CNT'].astype(int)
parcels_buildings_sdf.loc[(parcels_buildings_sdf['ADDR_CNT'].isnull() == True ), 'ADDR_CNT'] = 0
parcels_buildings_sdf['ADDR_CNT'] = parcels_buildings_sdf['ADDR_CNT'].astype(int)
parcels_buildings_sdf['PARCEL_ACRES'] = parcels_buildings_sdf['PARCEL_ACRES'].astype(float)

# create new count field and calculate
parcels_buildings_sdf['UNIT_COUNT'] = parcels_buildings_sdf['ADDR_CNT']


# fix single family (non-pud)
parcels_buildings_sdf.loc[(parcels_buildings_sdf['UNIT_COUNT'] == 0) & 
                 (parcels_buildings_sdf['TYPE_WFRC'] == 'single_family'), 
                 'UNIT_COUNT'] = 1

# fix duplex
parcels_buildings_sdf.loc[(parcels_buildings_sdf['UNIT_COUNT'] == 0) & 
                 (parcels_buildings_sdf['TYPE_WFRC'] == 'duplex'), 
                 'UNIT_COUNT'] = 2

parcels_buildings_sdf.loc[(parcels_buildings_sdf['SUBTYPE_WFRC'] == 'duplex'), 'UNIT_COUNT'] = 2

# fix apts
parcels_buildings_sdf.loc[(parcels_buildings_sdf['UNIT_COUNT'] == 0) & 
                 (parcels_buildings_sdf['PROP_CLASS'] == '115- 10+ UNITS'), 
                 'UNIT_COUNT'] = 10

parcels_buildings_sdf.loc[(parcels_buildings_sdf['UNIT_COUNT'] == 0) & 
                 (parcels_buildings_sdf['PROP_CLASS'] == '114- 5-9 UNITS'), 
                 'UNIT_COUNT'] = 5

parcels_buildings_sdf.loc[(parcels_buildings_sdf['UNIT_COUNT'] == 0) & 
                 (parcels_buildings_sdf['PROP_CLASS'] == '113- 3-4 UNITS'), 
                 'UNIT_COUNT'] = 3

###############################
# Subset to necessary columns
###############################

parcels_buildings_sdf = parcels_buildings_sdf[['COUNTY_NAME','COUNTY_ID','PARCEL_ID','TOTAL_MKT_VALUE',
                                               'LAND_MKT_VALUE','PARCEL_ACRES','UNIT_COUNT','BLDG_SQFT',
                                               'FLOORS_CNT','BUILT_YR','EFFBUILT_YR','max_dua','max_far','max_height', 
                                               'type1','type2','type3','type4','type5','type6','type7','type8', 
                                               'agriculture','basebldg','NoBuild','redev_friction','building_type_id','x','y',
                                               'SHAPE']].copy()



#############################################
# won't need these till all counties are done
#############################################

# create new REMM parcel IDs
parcels_buildings_sdf['parcel_id_remm'] = np.arange(len(parcels_buildings_sdf))

# copy table and only keep rows if basebldg = 1
buildings_df = parcels_buildings_sdf[parcels_buildings_sdf['basebldg'] == 1].copy()
buildings_df = buildings_df[['parcel_id_remm','BLDG_SQFT','TOTAL_MKT_VALUE', 'LAND_MKT_VALUE','UNIT_COUNT',
                             'building_type_id', 'FLOORS_CNT','BUILT_YR']].copy()

# calc building value?
buildings_df['B_VALUE'] = buildings_df['TOTAL_MKT_VALUE'] - buildings_df['LAND_MKT_VALUE']

# generate building id
buildings_df['building_id'] = np.arange(len(buildings_df))

# add new attributes
buildings_df['non_residential_sqft'] = 0
buildings_df['note'] = 'base'
buildings_df['residential_units'] = 0
buildings_df['unit_price_non_residential'] = 0
buildings_df['job_spaces'] = 0

buildings_df.loc[buildings_df['building_type_id'].isin([1,2])== False, 'non_residential_sqft'] = buildings_df['BLDG_SQFT']
buildings_df.loc[buildings_df['building_type_id'].isin([1,2])== True, 'residential_units'] = buildings_df['UNIT_COUNT']

# rename building columns
buildings_df = buildings_df.rename(columns={"BLDG_SQFT": "bldg_sqft", "FLOORS_CNT": "stories",'BUILT_YR':'year_built'})

In [ ]:
# export
parcels_buildings_sdf.spatial.to_featureclass(location= os.path.join(final, 'weber_parcels'),sanitize_columns=False)
parcels_buildings_sdf = parcels_buildings_sdf.drop(columns=['SHAPE', 'building_type_id'])
parcels_buildings_sdf.to_csv(os.path.join(outputs, 'weber_parcels.csv'), index=False)
buildings_df.to_csv(os.path.join(outputs, 'weber_buildings.csv'), index=False)